<a href="https://colab.research.google.com/github/migariane/DeltaMethodInfluenceFunction/blob/main/R-CodeBoxes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>